In [ ]:
import os
from openai import OpenAI
with open("../api.key", "r") as file:
    api_key = file.read().strip() 

os.environ["OPENAI_API_KEY"] = api_key


In [2]:
def call_llm(messages, response_format):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.0,
        max_tokens=256,
        top_p=1.0,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={
            "type": response_format
        }
    )
    return response.choices[0].message.content

In [3]:
labeling_instruction = """Please label each statement (the statements are seperated by a bullet point '- ') for truthfulness (True or False) based on the given context. 
Return the output in the JSON format where each statement is a key and the value is either True or False. The final format should be e.g.: {{'Mount Everest is a mountain': True, 'Mount Everest is the second highest mountain': False, 'Mount Everest is in the Himalaya': True}}"""

def get_prompt_labeling(mini_facts, ground_truth_source):
    messages = [{"role": "system", 
                "content" : [{"type": "text", 
                            "text":  f"{labeling_instruction}"}]},
                {"role": "user", "content": [{"type": "text", "text": f" Context: {ground_truth_source} \nStatements: {mini_facts}"}]}]
    return messages

In [4]:
mini_facts_instruction = f"""Your task is to breakdown claims/sentences into independant statements. 
You must NEVER correct or comment the original claims/sentences even if something of the original claims/sentences is incorrect.
Do NEVER generate statements that are not in the original claims/sentences. Every statement must start with an entity that specifies the topic (e.g. **The Fox Broadcasting Company** and not **The company**)."""
        
mini_facts_samples = ["The Hunger Games is a 2012 American science fiction film directed by John Peter and based on the novel of the same name by Suzanne Collins. Matt Lucena is an American former professional tennis player.",
"""Owen Wilson starred in the film "The Karate Kid" (2010) alongside martial arts expert Tom Wu. Owen Wilson voiced Lightning McQueen in the "Cars" franchise, not "The Royal Tenenbaums" franchise.""",
"Feels So Good is a song by the American R&B group Tony! Toni! TonÃ. The song was written by the group's lead vocalist Raphael Saadiq and producer Tony! Toni! TonÃ's lead vocalist Dwayne Wimberly."]
        
        
mini_facts_sample_outputs = ["""- **The Hunger Games** is a 2012 American science fiction film.
- **The Hunger Games** was directed by John Peter.
- **The Hunger Games** is based on a novel of the same name by Suzanne Collins.
- **Matt Lucena** is an American former professional tennis player.""",
"""- **Owen Wilson** starred in the film The Karate Kid (2010) alongside martial arts expert Tom Wu.
- **Owen Wilson** voiced Lightning McQueen in the Cars franchise.
- **Owen Wilson** did not voice Lightning McQueen in the The Royal Tenenbaums franchise.""",
"""- **Feels So Good** is a song by the American R&B group Tony! Toni! TonÃ.
- **Feels So Good** was written by the group's lead vocalist Raphael Saadiq and producer Tony! Toni! TonÃ's lead vocalist Dwayne Wimberly."""]

In [5]:
def get_prompt_mini_facts(gen_evidence):
    messages = [{"role": "system", 
            "content" : [{"type": "text", 
                        "text": f"{mini_facts_instruction}"}]},
        {"role": "user", "content": [{"type": "text", "text": f"{mini_facts_samples[0]}"}]},
        {"role": "assistant", "content": [{"type": "text", "text": f"{mini_facts_sample_outputs[0]}"}]},
        {"role": "user", "content": [{"type": "text", "text": f"{mini_facts_samples[1]}"}]},
        {"role": "assistant", "content": [{"type": "text", "text": f"{mini_facts_sample_outputs[1]}"}]},
        {"role": "user", "content": [{"type": "text", "text": f"{mini_facts_samples[2]}"}]},
        {"role": "assistant", "content": [{"type": "text", "text": f"{mini_facts_sample_outputs[2]}"}]},
        {"role": "user", "content": [{"type": "text", "text": f"{gen_evidence}"}]}]
    return messages

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


bart_model_path = "D:\huggingface\huggingface\hub\models--facebook--bart-large-mnli\snapshots\d7645e127eaf1aefc7862fd59a17a5aa8558b8ce"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Loading BART model...")
bart_model = AutoModelForSequenceClassification.from_pretrained(bart_model_path, local_files_only=True)
bart_tokenizer = AutoTokenizer.from_pretrained(bart_model_path, local_files_only=True)
bart_model.to(device)

def split_source_to_fit_with_hypothesis(source, hypothesis, bart_tokenizer, max_length=1024):
    """Splits the source into chunks so that each chunk, when combined with the hypothesis, fits within the token limit."""
    original_max_length = bart_tokenizer.model_max_length
    bart_tokenizer.model_max_length = int(1e12)  
    hypothesis_tokens = bart_tokenizer.encode(hypothesis, add_special_tokens=False)
    hypothesis_length = len(hypothesis_tokens)
    num_special_tokens = bart_tokenizer.num_special_tokens_to_add(pair=True)
    max_source_length = max_length - hypothesis_length - num_special_tokens
    if max_source_length <= 0:
        bart_tokenizer.model_max_length = original_max_length
        raise ValueError("The hypothesis is too long to fit within the max_length limit.")
    source_tokens = bart_tokenizer.encode(source, add_special_tokens=False)
    bart_tokenizer.model_max_length = original_max_length
    token_chunks = [source_tokens[i:i+max_source_length] for i in range(0, len(source_tokens), max_source_length)]
    text_chunks = [bart_tokenizer.decode(chunk, skip_special_tokens=True) for chunk in token_chunks]
    return text_chunks
    
def call_bart_model(source, statement):
    source_chunks = split_source_to_fit_with_hypothesis(source, statement, bart_tokenizer, max_length=1024)
    entailment_probs = []
    pred_labels = []
    for idx, chunk in enumerate(source_chunks):
        inputs = bart_tokenizer(
            chunk,
            statement,
            return_tensors='pt',
            truncation=True,
            max_length=1024,
            add_special_tokens=True
        )
        input_length = inputs['input_ids'].shape[1]
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bart_model(**inputs)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=1)
            dominating_class = probs.argmax(dim=1).item()

        class_names = ["Contradiction", "Neutral", "Entailment"]
        prob_entailment = probs[:, 2].item()
        entailment_probs.append(prob_entailment)
        pred_labels.append(class_names[dominating_class])

    filtered_labels = [label for label in pred_labels if label != "Neutral"]
    if filtered_labels:
        final_label = max(set(filtered_labels), key=pred_labels.count)
    else:
        final_label = max(set(pred_labels), key=pred_labels.count)
    return final_label

d:\MA\Finetuning\llama2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BART model...


In [15]:
import pandas as pd
import json 


df_evaluate = pd.read_pickle('application/whole_evidence.pkl')

df_evaluate['gen_evidence'] = df_evaluate['gen_evidence'].str.replace(" .###", ".").replace(". .", ".")
df_evaluate = df_evaluate[df_evaluate['gen_evidence'] != ' . .']
df_evaluate = df_evaluate[df_evaluate['gen_evidence'] != ' .']


df_true_bart_labels = pd.DataFrame()

for index, row in df_evaluate.iterrows():
    print(index)
    #mini_facts = "\n".join(row['Final Mini Facts'])
    #mini_facts = mini_facts.replace("- ", " ")
    #print(mini_facts)
    gen_evidence = row['gen_evidence']
    docs = row['docs']

    messages = get_prompt_mini_facts(gen_evidence)
    response = call_llm(messages, response_format="text")
    response = response.replace("**", "")

    ground_truth_source = row['Source']
    messages = get_prompt_labeling(response, ground_truth_source)
    response_format = "json_object"
    response = call_llm(messages, response_format)
    try:
        response_json = json.loads(response)
        response_data = [(key.strip(), int(value)) for key, value in response_json.items()]
    except Exception as e:
        print(f"Error parsing JSON at index {index}: {e}. The response is skipped.")
        continue

    for mini_fact, label in response_data:
        mini_fact = mini_fact.replace("- ", "")
        pred_label = call_bart_model(ground_truth_source, mini_fact)
        if pred_label == "Entailment" and label == 1:
            bart_label = 1
        elif pred_label == "Contradiction" and label == 0:
            bart_label = 0
        elif pred_label == "Entailment" and label == 0:
            #continue
            bart_label = 1
        elif pred_label == "Contradiction" and label == 1:
            bart_label = 0

        df_true_bart_labels = pd.concat([df_true_bart_labels, pd.DataFrame({'gen_evidence': [gen_evidence], 'docs': [docs], 'Source': [ground_truth_source], 'Mini Fact': [mini_fact], 'Label': [label], 'Bart Label': [bart_label]})])

# calculate accuracy 
df_true_bart_labels['Label'].sum() / len(df_true_bart_labels)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
Error parsing JSON at index 88: Unterminated string starting at: line 18 column 3 (char 922). The response is skipped.
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142


0.8480176211453745

In [16]:
df_true_bart_labels.to_pickle("application/whole_evidence_results.pkl")

NameError: name 'df_true_bart_labels' is not defined

In [4]:
import pandas as pd

df = pd.read_pickle("application/sentence_by_sentence.pkl")

In [5]:
df

,gen_evidence,Source,docs
0,Simon Fuller is a British entrepreneur and tel...,Guggenheim Partners is a global investment and...,"[Guggenheim Partners, Eldridge Industries]"
1,The Newcastle United athlete played as a strik...,Cheick Ismaël Tioté (] ; 21 June 1986 – 5 June...,"[FC Bibo, Cheick Tioté]"
2,"The company, which holds that Robert Melville ...",Robert Melville is a British car designer curr...,"[Jaguar Land Rover, Robert Melville (car desig..."
3,Black Nativity is a 2013 American musical dram...,Nasir Bin Olu Dara Jones ( ; born September 14...,"[Black Nativity (film), Nas]"
4,.### .###,Ali Mohammad Zafar (born 18 May 1980) is a Pak...,"[Syed Mohammad Ahmed, Tere Bin Laden, Ali Zafar]"
...,...,...,...
138,The Canadian Museum of Flight is located in La...,The Canadian Museum of Flight (formally the Ca...,"[Helicopter, Canadian Museum of Flight, Langle..."
139,"The documentary film ""Woodstock"" captured the ...","American Dream is a 1990 ""cinéma vérité"" docum...","[American Dream (film), Woodstock (film), Mich..."
140,The town of Ogunquit is located in the state o...,"Ogunquit is a town in York County, Maine, Unit...","[Agamenticus, Ogunquit]"
141,Paul Sloane is a film director .### Paul Sloan...,"Paul Sloane (April 19, 1893 – November 15, 196...","[Jacques Demy, Paul Sloane (director), Break o..."


In [17]:
import pandas as pd

df = pd.read_pickle("application/baseline.pkl")

df['gen_evidence'].tolist()

['Mark Walter is the CEO of Guggenheim Partners, and Simon Fuller is the founder of XIX Entertainment, a global entertainment company.',
 'The Newcastle United athlete that came through FC Bibo played as a striker.',
 'The company, which holds that Robert Melville previously worked with, is a global leader in the design, development, manufacture and supply of high-quality, innovative products. Robert Melville is a former executive of the company.',
 'The film Black Nativity is a 2013 American musical drama film directed by Kasi Lemmons, starring Angela Bassett, Forest Whitaker, and Mary J. Blige, not Nas.',
 'The movie "Silk" (2011) is a Pakistani drama film that won five Lux Style Awards, including Best Film, Best Director, Best Actor, Best Actress, and Best Actor in a Supporting Role. Syed Mohammad Ahmed is a Pakistani television and film writer, director, and producer.',
 'The movie was "The Miracle of Bernabéu" (2003), a documentary film about the 1980 European Cup Final between No

In [40]:
df

,claim,docs,ground_truth_source
0,Simon Fuller former position at the firm where...,"[Guggenheim Partners, Eldridge Industries]",Guggenheim Partners is a global investment and...
1,The Newcastle United athlete that came through...,"[FC Bibo, Cheick Tioté]",Cheick Ismaël Tioté (] ; 21 June 1986 – 5 June...
2,"design, development, manufacture is the princi...","[Jaguar Land Rover, Robert Melville (car desig...",Robert Melville is a British car designer curr...
3,Black Nativity is a 2013 American musical dram...,"[Black Nativity (film), Nas]",Nasir Bin Olu Dara Jones ( ; born September 14...
4,The pop singer in the lead role received five ...,"[Syed Mohammad Ahmed, Tere Bin Laden, Ali Zafar]",Ali Mohammad Zafar (born 18 May 1980) is a Pak...
...,...,...,...
138,A helicopter is a type of aircraft that can be...,"[Helicopter, Canadian Museum of Flight, Langle...",The Canadian Museum of Flight (formally the Ca...
139,"The 1969 festival production, which was direct...","[American Dream (film), Woodstock (film), Mich...","American Dream is a 1990 ""cinéma vérité"" docum..."
140,The Mount Agamenticus region covers the town o...,"[Agamenticus, Ogunquit]","Ogunquit is a town in York County, Maine, Unit..."
141,Paul Sloane is not noted for directing more th...,"[Jacques Demy, Paul Sloane (director), Break o...","Paul Sloane (April 19, 1893 – November 15, 196..."


In [54]:
df = pd.read_pickle("application/application_data.pkl")

#print(df['Label'].sum() / len(df))

for index, row in df.iterrows():
    print(row['claim'])
    #print(row['ground_truth_source'])
    #print(row['Mini Fact'])
    #print(row['Label'])

Simon Fuller former position at the firm where Mark Walter is the CEO was president.
The Newcastle United athlete that came through FC Bibo played as a defensive midfielder.
design, development, manufacture is the principal activity of the company which holds that Robert Melville previously worked with.
Black Nativity is a 2013 American musical drama film directed by Kasi Lemmons - the film stars an African American ensemble cast featuring the American hip hop recording artist, record producer and actor and entrepreneur better known by his stage name Nas.
The pop singer in the lead role received five Lux Style Awards. Syed Mohammad Ahmed wrote the dialogue for this movie.
Peter Rufai was the Italian goalkeeper that participated in a football game for a movie premiere. The movie was the first movie aggregated on the CREETIQ website.
A helicopter is a type of aircraft that can be used in congested or isolated areas and is used at Langley Regional Airport.
The founder of the Metroplex rec

In [63]:
df = pd.read_pickle("application/baseline_results.pkl")

#print(df['Label'].sum() / len(df))

for index, row in df.iterrows():
    #print(row['gen_evidence'])
    #print(row['ground_truth_source'])
    print(row['Mini Fact'])
    print(row['Label'])


Mark Walter is the CEO of Guggenheim Partners.
1
Simon Fuller is the founder of XIX Entertainment.
0
XIX Entertainment is a global entertainment company.
1
The Newcastle United athlete that came through FC Bibo played as a striker.
0
The company is a global leader in the design, development, manufacture, and supply of high-quality, innovative products.
1
Robert Melville previously worked with the company.
1
Robert Melville is a former executive of the company.
0
The film Black Nativity is a 2013 American musical drama film.
1
The film Black Nativity was directed by Kasi Lemmons.
1
The film Black Nativity stars Angela Bassett, Forest Whitaker, and Mary J. Blige.
1
The film Black Nativity does not star Nas.
0
The movie Silk (2011) is a Pakistani drama film
0
The movie Silk won five Lux Style Awards
0
The movie Silk won the Lux Style Award for Best Film
0
The movie Silk won the Lux Style Award for Best Director
0
The movie Silk won the Lux Style Award for Best Actor
0
The movie Silk won t

The English disc jockey who hosted "The Voice UK" is Marvin Humes, born on 18 March 1985. Marvin Humes is an English disc jockey and television presenter, best known for hosting the British reality television show "The Voice UK" since its inception in 2012.'

Marvin Humes is an English disc jockey, Marvin Humes was born on 18 March 1985, Marvin Humes is a disc jockey and radio host, Marvin Humes is best known as a former member of the boyband JLS

The English disc jockey was born on 18 March 1985, The English disc jockey is best known for hosting the late show across the UK\'s Capital FM Network, and the Vodafone Big Top 40 chart show across UK commercial radio stations on a Sunday afternoon .### Marvin Humes is the English disc jockey who hosts "The Voice UK", Marvin Humes was born on 18 March 1985 .###',

In [58]:
import pandas as pd

df = pd.read_pickle("application/sentence_by_sentence_results.pkl")

for index, row in df.iterrows():
    print(row['gen_evidence'])
    print(row['Mini Fact'])
    print(row['Label'])
    print(row['Bart Label'])
    print("\n")

Simon Fuller is a British entrepreneur and television producer, Mark Walter is the CEO of Guggenheim Partners. Simon Fuller is a British entrepreneur and television producer.
Simon Fuller is a British entrepreneur and television producer.
1
0


Simon Fuller is a British entrepreneur and television producer, Mark Walter is the CEO of Guggenheim Partners. Simon Fuller is a British entrepreneur and television producer.
Mark Walter is the CEO of Guggenheim Partners.
1
1


The Newcastle United athlete played as a striker, The Newcastle United athlete came through F,C, Bibó. FC Bibó is an Ivorian semi-professional football team based in Kossou, Yamoussoukro, Ivory Coast.
The Newcastle United athlete played as a striker.
0
0


The Newcastle United athlete played as a striker, The Newcastle United athlete came through F,C, Bibó. FC Bibó is an Ivorian semi-professional football team based in Kossou, Yamoussoukro, Ivory Coast.
The Newcastle United athlete came through F.C. Bibó.
1
1


The Newcas

In [57]:
import pandas as pd

df = pd.read_pickle("application/whole_evidence.pkl")

df['gen_evidence'].tolist()

['Simon Fuller is the founder of XIX Entertainment, a global entertainment company .###',
 'The Newcastle United athlete played as a striker, The Newcastle United athlete came through F,C, Bibó .###',
 'The company, which holds that Robert Melville previously worked with, is Jaguar Land Rover, Robert Melville is a British car designer currently working as Design Director at McLaren Automotive .###',
 'Black Nativity is a 2013 American musical drama film, Black Nativity stars Nas .###',
 'Syed Mohammad Ahmed is a Pakistani film director .###',
 'The Miracle of Bernabéu is a documentary film about the 1980 European Cup Final between Nottingham Forest and Hamburg, Peter Rufai is a Nigerian retired footballer who played as a goalkeeper .###',
 'Langley Regional Airport is a small airport located in Langley, British Columbia, Canada, Helicopters are commonly used in congested or isolated areas due to their ability to take off and land vertically .###',
 'The Metroplex record label was found

In [52]:
import pandas as pd

df = pd.read_pickle("application/whole_evidence.pkl")

df['gen_evidence'].tolist()

['Simon Fuller is the founder of XIX Entertainment, a global entertainment company .###',
 'The Newcastle United athlete played as a striker, The Newcastle United athlete came through F,C, Bibó .###',
 'The company, which holds that Robert Melville previously worked with, is Jaguar Land Rover, Robert Melville is a British car designer currently working as Design Director at McLaren Automotive .###',
 'Black Nativity is a 2013 American musical drama film, Black Nativity stars Nas .###',
 'Syed Mohammad Ahmed is a Pakistani film director .###',
 'The Miracle of Bernabéu is a documentary film about the 1980 European Cup Final between Nottingham Forest and Hamburg, Peter Rufai is a Nigerian retired footballer who played as a goalkeeper .###',
 'Langley Regional Airport is a small airport located in Langley, British Columbia, Canada, Helicopters are commonly used in congested or isolated areas due to their ability to take off and land vertically .###',
 'The Metroplex record label was found

In [8]:
for index, row in df.iterrows():
    print(row['gen_evidence'])
    print("\n")

Mark Walter is the CEO of Guggenheim Partners, and Simon Fuller is the founder of XIX Entertainment, a global entertainment company.


The Newcastle United athlete that came through FC Bibo played as a striker.


The company, which holds that Robert Melville previously worked with, is a global leader in the design, development, manufacture and supply of high-quality, innovative products. Robert Melville is a former executive of the company.


The film Black Nativity is a 2013 American musical drama film directed by Kasi Lemmons, starring Angela Bassett, Forest Whitaker, and Mary J. Blige, not Nas.


The movie "Silk" (2011) is a Pakistani drama film that won five Lux Style Awards, including Best Film, Best Director, Best Actor, Best Actress, and Best Actor in a Supporting Role. Syed Mohammad Ahmed is a Pakistani television and film writer, director, and producer.


The movie was "The Miracle of Bernabéu" (2003), a documentary film about the 1980 European Cup Final between Nottingham For

In [11]:
import pandas as pd

df = pd.read_pickle("application/sentence_by_sentence_results.pkl")

In [14]:
for index, row in df.iterrows():
    print(row['gen_evidence'])
    print(row['Mini Fact'])
    print("\n")

Simon Fuller is a British entrepreneur and television producer, Mark Walter is the CEO of Guggenheim Partners. Simon Fuller is a British entrepreneur and television producer.
Simon Fuller is a British entrepreneur and television producer.


Simon Fuller is a British entrepreneur and television producer, Mark Walter is the CEO of Guggenheim Partners. Simon Fuller is a British entrepreneur and television producer.
Mark Walter is the CEO of Guggenheim Partners.


The Newcastle United athlete played as a striker, The Newcastle United athlete came through F,C, Bibó. FC Bibó is an Ivorian semi-professional football team based in Kossou, Yamoussoukro, Ivory Coast.
The Newcastle United athlete played as a striker.


The Newcastle United athlete played as a striker, The Newcastle United athlete came through F,C, Bibó. FC Bibó is an Ivorian semi-professional football team based in Kossou, Yamoussoukro, Ivory Coast.
The Newcastle United athlete came through F.C. Bibó.


The Newcastle United athle

In [19]:
len(df_true_bart_labels)

523

In [20]:
factuality_score = df_true_bart_labels['Label'].mean()
factuality_score

0.5353728489483748

In [18]:
df_bart = df_true_bart_labels.loc[df_true_bart_labels['Label'] == df_true_bart_labels['Bart Label']]

df_bart['Label'].sum() / len(df_bart)

0.8976377952755905

In [20]:
df_evaluate = pd.read_pickle('application/sentence_by_sentence_results.pkl')

In [24]:
df_evaluate.drop_duplicates(subset=['Mini Fact'], inplace=True)
df_evaluate['Label'].sum() / len(df_evaluate)

0.7922606924643585

In [23]:
df_evaluate

,gen_evidence,docs,Source,Mini Fact,Label,Bart Label
0,Simon Fuller is a British entrepreneur and tel...,"[Guggenheim Partners, Eldridge Industries]",Guggenheim Partners is a global investment and...,Simon Fuller is a British entrepreneur and tel...,1,0
0,Simon Fuller is a British entrepreneur and tel...,"[Guggenheim Partners, Eldridge Industries]",Guggenheim Partners is a global investment and...,Mark Walter is the CEO of Guggenheim Partners.,1,1
0,The Newcastle United athlete played as a strik...,"[FC Bibo, Cheick Tioté]",Cheick Ismaël Tioté (] ; 21 June 1986 – 5 June...,The Newcastle United athlete played as a striker.,0,0
0,The Newcastle United athlete played as a strik...,"[FC Bibo, Cheick Tioté]",Cheick Ismaël Tioté (] ; 21 June 1986 – 5 June...,The Newcastle United athlete came through F.C....,1,1
0,The Newcastle United athlete played as a strik...,"[FC Bibo, Cheick Tioté]",Cheick Ismaël Tioté (] ; 21 June 1986 – 5 June...,FC Bibó is an Ivorian semi-professional footba...,1,1
...,...,...,...,...,...,...
0,"KCTV airs a variety of programming, including ...","[Check, Please!, KCPT, KCTV]","Check, Please! is a popular, multi-Emmy Award ...","KCTV airs a variety of programming, including ...",1,1
0,"KCTV airs a variety of programming, including ...","[Check, Please!, KCPT, KCTV]","Check, Please! is a popular, multi-Emmy Award ...",KCTV is a television station licensed to Kansa...,1,1
0,"KCTV airs a variety of programming, including ...","[Check, Please!, KCPT, KCTV]","Check, Please! is a popular, multi-Emmy Award ...",KCTV is owned by the Meredith Local Media subs...,1,0
0,"KCTV airs a variety of programming, including ...","[Check, Please!, KCPT, KCTV]","Check, Please! is a popular, multi-Emmy Award ...",KCTV does air the program 'Check Please!'.,0,0


In [ ]:
df_evaluate = pd.read_pickle('application/sentence_by_setnence_results.pkl')

df_evaluate['gen_evidence'] = df_evaluate['gen_evidence'].str.replace(" .###", ".")
df_evaluate['gen_evidence'] = df_evaluate['gen_evidence'].str.replace(". .", ".")
df_evaluate = df_evaluate[df_evaluate['gen_evidence'] != ' . .']
df_evaluate = df_evaluate[df_evaluate['gen_evidence'] != ' .']

for index, row in df_evaluate.iterrows():
    print(row['gen_evidence'])

In [ ]:
'The English disc jockey was born on 18 March 1985.',
 "The English disc jockey is best known for hosting the late show across the UK's Capital FM Network.",
 'The English disc jockey is best known for hosting the Vodafone Big Top 40 chart show across UK commercial radio stations on a Sunday afternoon.',
 'Marvin Humes is the English disc jockey who hosts The Voice UK.',
 'Marvin Humes was born on 18 March 1985.',

The English disc jockey was born on 18 March 1985,
The English disc jockey is best known for hosting the late show across the UK's Capital FM Network, 
 and the Vodafone Big Top 40 chart show across UK commercial radio stations on a Sunday afternoon. 
Marvin Humes is the English disc jockey who hosts "The Voice UK", Marvin Humes was born on 18 March 1985.

In [ ]:
df_true_bart_labels.to_pickle('application/application_scenario1_factuality.pkl')

In [ ]:
df = pd.read_pickle('application.pkl')
#df_test_with_claims = pd.read_pickle("test_hover.pkl")

In [ ]:
#df_with_claims = pd.merge(df, df_test_with_claims[['gen_evidence','claim']], on='gen_evidence', how='inner')

In [ ]:
df_application = 

In [ ]:
df_with_claims = pd.merge(df, df_claims[['gen_evidence','claim']], on='gen_evidence', how='inner')

In [ ]:
df_with_claims.to_pickle('application/application_with_claims.pkl')

In [ ]:
for gen_e in df['gen_evidence'].tolist():
    if gen_e in df_claims['gen_evidence'].tolist():
        print(gen_e)

In [ ]:
import pandas as pd

